In [2]:
import pandas as pd
import numpy as np
import os

# Series

In [ ]:
import pandas as pd
import numpy as np
# 构造序列
series1 = pd.Series([2.8,3.01,8.99,8.59,5.18])
series2 = pd.Series([2.8,3.01,8.99,8.59,5.18],index = ['a','b','c','d','e'],name ='这是一个series')
series3 = pd.Series({'北京':2.8,'上海':3.01,'广东':8.99,'江苏':8.59,'浙江':5.18})
#series方法
series2.values
series3.index

# dafaframe

In [ ]:
# dafaframe其实就是一个二维表结构，是数据分析中，最常用的数据结构
list1 = [['张三',23,'男'],['李四',27,'女'],['王二',26,'女']]#使用嵌套列表
df1 = pd.DataFrame(list1,columns=['姓名','年龄','性别'])
type(df1)
df2 = pd.DataFrame({'姓名':['张三','李四','王二'],'年龄':[23,27,26],'性别':['男','女','女']}) #使用字典,字典的键被当成列名
df2
array1 = np.array([['张三',23,'男'],['李四',27,'女'],['王二', 26,'女']]) #使用numpy
df3 = pd.DataFrame(array1,columns=['姓名','年龄','性别'],index = ['a','b','c'] )
#dataframe方法
df2.to_numpy()
df2.index
df2.columns
df2.dtypes
df2.ndim
df2.size

In [ ]:
df = pd.read_csv('data/baby_trade_history.csv', encoding='utf-8',dtype={'user_id':str})
# 查看数据
df.info()
df.head(5)  # 查看前5行
df.tail(5)  # 查看后5行
df.columns  # 查看变量名称

In [ ]:
# 简单索引
df['user_id']
df['user_id'][1:5]  # 第二行到第五行
# 多个变量选择
df[['user_id','buy_mount','day']][:5]   

In [ ]:
df.loc[3:4] # 选择行索引标签
df.loc[:,['user_id','buy_mount']]   # 选择某两列
df.loc[1:3,['user_id','buy_mount']] # loc在这里选择的是行索引标签
df.loc[df.user_id =='786295544',['user_id','buy_mount','day']]
df.loc[(df.user_id =='786295544') | (df.user_id =='444069173'),['user_id','buy_mount','day']]   # 多个条件选择

In [ ]:
df.iloc[:,1:4]  # 按照位置来选择第二列到第四列
df.iloc[:,[0,2]]    # 按照位置来选择第1列和第3列
df.iloc[3,[1,2]]    # 选择第4行，第2列和第3列数据, 这里的3代表的不是索引标签而是位置
df.iloc[2:7,[1,2]]  # 选择第3行到第7行，第2列和第3列数据

In [ ]:
# loc和iloc的区别
print(df.loc[2:7])
print(df.iloc[2:7])

In [ ]:
#增加一列,购买量,购买量超过3的为高，低于3的为底
df['购买量'] = np.where(df['buy_mount'] >3,'高','低')
df

In [ ]:
# 将这一列取出来，赋值给对象auction_id,然后在数据中删除这一列，再将其添加进去
auction_id = df['auction_id']
del df['auction_id']
print(df.head(1))
df.insert(0, 'auction_id', auction_id)
df.head(1)

In [ ]:
import pandas as pd
df = pd.read_csv('data/ships.csv')
print(df[df.isnull().any(axis=1)])
print(df.iloc[-1]['name'])
df['name'] = df['name'].str.strip()
print(df.iloc[-1]['name'])
import numpy as np
mask = df['name'].str.strip() == ''
df.loc[mask, 'name'] = np.nan
print(df[df.isnull().any(axis=1)])

In [ ]:
import pandas as pd


df = pd.read_csv('data/cart.csv', parse_dates=['date'])
print(df)

df['amount'].fillna(1, inplace=True)
print(df)

most_common = df['name'].mode()[0]
df['name'].fillna(most_common, inplace=True)
print(df)

df['date'].fillna(method='ffill', inplace=True)
print(df)

import numpy as np
prices = df.groupby('name')['price'].transform(np.mean)
print(prices)

df['price'].fillna(prices, inplace=True)
print(df)

### 行操作

In [ ]:
import pandas as pd

df = pd.read_csv('data/metrics.csv', parse_dates=['time'])
print(df)

df = pd.melt(
    df,
    value_vars=['cpu', 'memory'],
    id_vars=['time'],
    var_name='metric',
)
print(df)

## 列操作

In [ ]:
import pandas as pd

df = pd.read_csv('data/points.csv')
print(df.dtypes)

def asint(val):
    return int(val, base=0)

df['color'] = df['color'].apply(asint)
print(df.dtypes)

bools = {
    'yes': True,
    'no': False,
}
df['visible'] = df['visible'].map(bools)
df.dtypes

df

In [ ]:
import pandas as pd

csv_file = '2021-06.csv'
path = 'data/'
df = pd.read_csv(path+csv_file)
print(df)
df['date'] = csv_file[:-len('.csv')]
print(df)
times = df['time'].str.split('-', expand=True)
times.columns = ['start', 'end']
print(times)
df = pd.concat([df, times], axis=1)
print(df)

df['start'] = pd.to_datetime(
    df['date'].str.cat(df['start'], sep='T')
)
df['end'] = pd.to_datetime(
    df['date'].str.cat(df['end'], sep='T')
)
print(df)

print((df['end'] - df['start']).sum())

In [ ]:
import pandas as pd

df = pd.read_csv('data/rides.csv')
print(df)
# 评估描述 DataFrame 列操作的字符串。这就允许 eval 运行任意代码，如果将用户输入传递给此函数，就可能导致代码注入。
mask = df.eval('name.isnull() | distance <= 0')
print(mask)

df = df[~mask]
print(df)

## 数据帧范式验证

In [ ]:
import pandas as pd

df = pd.read_csv('data/ships.csv')

import pandera as pa

# 设定数据帧范式
schema = pa.DataFrameSchema({
    # 创建列验证器对象
    'name': pa.Column(pa.String),
    'lat': pa.Column(pa.Float),
    'lng': pa.Column(pa.Float),
})

# 根据模式规范验证数据帧
schema.validate(df)

In [ ]:
schema = pa.DataFrameSchema({
    'name': pa.Column(pa.String),
    'lat': pa.Column(
        pa.Float,
        nullable=True,
        checks=pa.Check(
            lambda v: v >= -90 and v <= 90,
            element_wise=True,
        ),
    ),
    'lng': pa.Column(
        pa.Float,
        nullable=True,
        checks=pa.Check(
            lambda v: v >= -180 and v <= 180,
            element_wise=True,
        ),
    ),
})

schema.validate(df)

In [ ]:
import pandas as pd
import numpy as np

size = 5
generator = np.random.default_rng(5)
df = pd.DataFrame({
    'time': pd.date_range('2021', freq='17s', periods=size),
    'name': ['cpu'] * size,
    'value': generator.standard_normal() * 40,
})

import pyarrow as pa

schema = pa.schema([
    ('time', pa.timestamp('ms')),
    ('name', pa.string()),
    ('value', pa.float64()),
])
print(type(df['time'][0]))

out_file = 'data/metrics.parquet'
# 将数据帧写入二进制 parquet 格式。
df.to_parquet(out_file, schema=schema)
print(pd.read_parquet(out_file))

df['time'] = df['time'].astype(str)
print(type(df['time'][0]))

# 转换格式后schema校验将报ArrowTypeError
# df.to_parquet(out_file, schema=schema)

In [ ]:
import pandas as pd

df = pd.read_csv('data/orders.csv', parse_dates=['time'])

# 行校验
def is_valid_row(row):
    if row['time'] < pd.Timestamp('1900'):
        return False
    if pd.isnull(row['symbol']) or row['symbol'].strip() == '':
        return False
    if row['price'] <= 0:
        return False
    return True

ok_df = df[df.apply(is_valid_row, axis=1)]

num_bad = len(df) - len(ok_df)
percent_bad = num_bad/len(df) * 100
print(f'{percent_bad:.2f}% bad rows')
if num_bad > 0:
    bad_rows = df[~df.index.isin(ok_df.index)]
    print('bad rows:')
    print(bad_rows)

## 删除

In [ ]:
# 同时删除多个变量，需要以列表的形式
# labels表示删除的数据, axis表示作用轴，inplace=True表示是否对原数据生效,
# axis=0按行操作, axis=1按列操作
df.drop(labels = ['property', '购买量'],axis = 1)   # 1代表沿着列的方向
# 按行删除法
df.drop(labels = [3,4], axis= 0)    # 删除索引标签3和4对应的行
df.drop(labels= range(6,11), axis=0)    # 删除索引名称1到10,注意range迭代器产生的是1到10
# 查看
df
# 默认情况下，原始DataFrame保持不变，并返回一个新的DataFrame。如果参数inplace设置为True，则将更改原始DataFrame。在这种情况下，不会返回任何新的DataFrame，并且返回值为None。

In [ ]:
df = pd.read_csv('data/baby_trade_history.csv', encoding='utf-8',dtype={'user_id':str},index_col=0)
# 按行名指定（行标签）删除行，行标签所在列为index_col
df.drop(labels= ['307215255'], axis=0)

In [ ]:
# 多行多列的删除
df.drop(index=['532110457', '786295544', '917056007'],
              columns=['buy_mount', 'cat1'])

In [ ]:
df.drop(index=df.index[[1, 3, 5]],
              columns=df.columns[[1, 2]])

## 数据修改和查找

In [ ]:
df1 = pd.read_csv('data/sam_tianchi_mum_baby.csv',encoding = 'utf-8',dtype =str)
print(df1.head(1))
# 将gender为0的改为女性、1改为男性、2改为未知
df1.loc[df1['gender'] =='0','gender'] ='女性'
df1.loc[df1['gender'] =='1','gender'] ='男性'
df1.loc[df1['gender'] =='2','gender'] ='未知'
df1.head(10)

In [ ]:
# 修改列标签和行索引名称
df1.rename(columns = {'user_id':'用户ID','birthday':'出生日期','gender':'性别'},inplace = True)
df1.rename(index = {1:'one',10:'ten' },inplace = True) # 修改行索引名称
df1.reset_index(drop=True,inplace=True) # 重置索引
df1.head(5)

## 查询

In [ ]:
# 条件查询
df[df.buy_mount > 3]    # 性别等于未知
df[~(df.buy_mount > 3)] # ~代表非
df[(df.buy_mount > 3) & (df.day > 20140101)]    # 多条件查询
# 使用isin()方法
df[df['auction_id'].isin([41098319944, 17916191097,21896936223])]

## 合并

In [ ]:
# pip install xlrd -i https://pypi.tuna.tsinghua.edu.cn/simple 
# 纵向堆叠可以理解为把不同的表，字段名称一样，整合在一起
# 关联字段必须类型一致
df = pd.read_csv('data/baby_trade_history.csv', encoding='utf-8',dtype={'user_id':str}) # 交易数据
df1 = pd.read_csv('data/sam_tianchi_mum_baby.csv',encoding = 'utf-8',dtype =str)    # 婴儿信息
df2 = pd.merge(left = df, right=df1,  how='inner',  left_on='user_id', right_on = 'user_id', validate="many_to_many")    # 内连接
df2.head(10)

In [ ]:
import pandas as pd

df = pd.read_csv('data/heights.csv')
max_heights = pd.DataFrame([[1, 32],], columns=['grade', 'max_height'])
print(max_heights)

mdf = pd.merge(df, max_heights, how='left', on=None, validate="many_to_many")
print(mdf)

df[mdf['height'] > mdf['max_height']]

## 统计

In [ ]:
df = pd.read_csv('data/online_order.csv',encoding = 'gbk',dtype={'customer':str,'order':str})
df.dtypes

In [ ]:
df.head(1)  # 查看数据
grouped = df.groupby('weekday') # 创建分组对象,按照星期进行分组
grouped.sum()['total_items']    # 计算不同的星期，商品数量的总和
grouped = df.groupby(by =['customer','weekday'])    # 创建分组对象,按照用户和星期
grouped.sum()['total_items'].head(50)   # 调用方法,计算不同的用户周一到周天的订购商品数量的总和

## 实操

### 定位不良记录

In [ ]:
import pandas as pd
df = pd.read_csv('data/rides.csv')
# 缺失值记录
null_mask = df.isnull().any(axis=1)
# plate少于3个大写字母或数字的组合
plate_mask = ~df['plate'].str.match(r'^[0-9A-Z]{3,}', na=False)
# distance小于0
dist_mask = df['distance'] < 0
# 合并所有有不良值的记录
mask = null_mask | plate_mask | dist_mask
print(df[mask])

### 数据修复

In [ ]:
import pandas as pd

df = pd.read_csv('data/workshops.csv')
print(df)

"""
Fix the data frame. At the end, row should have the following columns:
- start: pd.Timestemap
- end: pd.Timestamp
- name: str
- topic: str (python or go)
- earnings: np.float64
"""

# Fill Year & Month
df['Year'].fillna(method='ffill', inplace=True)
df['Month'].fillna(method='ffill', inplace=True)
print(df)

# 仅保留 earnings 非空
df = df[pd.notnull(df['Earnings'])].copy()
print(df)

# %%
def as_date(row, col):
    year = int(row['Year'])
    month = row['Month']
    day = int(row[col])
    ts = f'{month} {day}, {year}'
    return pd.to_datetime(ts, format='%B %d, %Y')

df['start'] = df.apply(as_date, axis=1, args=('Start',))
df['end'] = df.apply(as_date, axis=1, args=('End',))
print(df)

# %% 根据关键定 Extract topic
def topic(name):
    if 'go' in name:
        return 'go'
    if 'python' in name:
        return 'python'

df['topic'] = df['Name'].str.lower().apply(topic)
print(df)

# %% Earnings
import numpy as np
df['earnings'] = pd.to_numeric(
    df['Earnings'].str.replace(r'[$,]', '')
).astype(np.float64)
print(df)

# %% Cleanup
df = df[['start', 'end', 'Name', 'topic', 'earnings']]
df.rename(columns={'Name': 'name'}, inplace=True)
print(df)

# Categorical

pandas.Categorical(val, categories = None, ordered = None, dtype = None) ： 表示一个分类变量。
Categorical 是一种 pandas 数据类型，与统计中的分类变量相对应。此类变量具有固定且数量有限的可能值。例如，成绩、性别、血型等。
此外，对于分类变量，逻辑顺序与分类数据（如 “一”、“二”、“三”）不同。但这些变量的排序使用逻辑顺序。

[courses](https://www.geeksforgeeks.org/python-pandas-categorical/)

In [ ]:
import numpy as np
import pandas as pd
 
# Categorical using dtype
c = pd.Series(["a", "b", "d", "a", "d"], dtype ="category")
print ("\nCategorical without pandas.Categorical() : \n", c)
 
 
c1 = pd.Categorical([1, 2, 3, 1, 2, 3])
print ("\n\nc1 : ", c1)
 
c2 = pd.Categorical(['e', 'm', 'f', 'i',
                     'f', 'e', 'h', 'm' ])
print ("\nc2 : ", c2)

c3 = pd.Categorical(['e', 'm', 'f', 'i',
                     'f', 'e', 'h', 'm' ], ordered = True)
print ("\nc3 : ", c3)

# Mixed categories
c4 = pd.Categorical(['a', 2, 3, 1, 2, 3])
print ("\nc4 : ", c4)
 
c5 = pd.Categorical(['a', 2, 3, 1, 2, 3], ordered = True)
print ("\nc5 : ", c5)

# using categories attribute
c6 = pd.Categorical([1, 2, 3, 1, 2, 3], categories = [4, 1, 3, 5])
print ("\nc6 : ", c6)
 
print("\n\nSeries : \n", pd.Series(c6))
 
df = pd.DataFrame({"A":[1, 2, 3, 1, 2, 3]})
df["B"] = c6
print ("\n\nDataframe : \n", df)